### Importing Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

In [5]:
# Importing cleaned dataset and dropping old index

retail = pd.read_csv('./dataset/retail.csv')
retail = retail.drop('Unnamed: 0', axis=1)

In [ ]:
# Distributions
fig, axs = plt.subplots(nrows=2, ncols=3, figsize = (20,25))

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=45)

axs[0, 0].hist(retail['Product line'], color = 'lightblue', bins = 11)
axs[0, 0].set_xlabel('Product Type')
axs[0, 0].set_ylabel('Frequency')
axs[0, 0].set_title('Product Line Distribution')

axs[0, 1].hist(retail_original['Payment'], color = 'silver')
axs[0, 1].set_xlabel('Payment type')
axs[0, 1].set_ylabel('Frequency')
axs[0, 1].set_title('Payment Distribution')

axs[0, 2].hist(retail_original['Rating'], color = 'wheat', bins = 20)
axs[0, 2].axvline(retail_original['Rating'].mean(), color='k', linestyle='dashed', linewidth=1)
axs[0, 2].set_xlabel('Rating')
axs[0, 2].set_ylabel('Frequency')
axs[0, 2].set_title('Rating Distribution')

axs[1, 0].hist(retail_original['Branch'], color = 'salmon', bins = 11)
axs[1, 0].set_xlabel('Branch')
axs[1, 0].set_ylabel('Frequency')
axs[1, 0].set_title('Branch Distribution')

axs[1, 1].hist(retail_original['Unit price'], color = 'teal', bins = 20)
axs[1, 1].axvline(retail_original['Unit price'].mean(), color='k', linestyle='dashed', linewidth=1)
axs[1, 1].set_xlabel('Unit price')
axs[1, 1].set_ylabel('Frequency')
axs[1, 1].set_title('Unit price Distribution')

axs[1, 2].hist(retail_original['Hour'], color = 'lavender', bins = 10)
axs[1, 2].set_xlabel('Hour')
axs[1, 2].set_ylabel('Frequency')
axs[1, 2].set_title('Hour Distribution')

plt.show()